# Basics
Let's start with some basics, we will use the following objects as the base for the rest of the tutorial.

In [1]:
from napalm_base import get_network_driver
import napalm_yang

import json

junos_configuration = {
    'hostname': '127.0.0.1',
    'username': 'vagrant',
    'password': '',
    'optional_args': {'port': 12203, 'config_lock': False}
}

eos_configuration = {
    'hostname': '127.0.0.1',
    'username': 'vagrant',
    'password': 'vagrant',
    'optional_args': {'port': 12443}
}

junos = get_network_driver("junos")
junos_device = junos(**junos_configuration)

eos = get_network_driver("eos")
eos_device = eos(**eos_configuration)

def pretty_print(dictionary):
    print(json.dumps(dictionary, sort_keys=True, indent=4))

# Creating a Binding

To work with YANG models you first create a root object with  ``napalm_yang.BaseBinding()`` and then you add as many models as you want with the `add_model` method:

In [2]:
config = napalm_yang.base.Root()

# Adding models to the object
config.add_model(napalm_yang.models.openconfig_interfaces())
config.add_model(napalm_yang.models.openconfig_vlan())

At this point, you can use the method ``model_to_dict()`` to visualize the binding and the attached models:

In [3]:
# Printing the model in a human readable format
pretty_print(napalm_yang.utils.model_to_dict(config))

{
    "openconfig-interfaces:interfaces [rw]": {
        "interface [rw]": {
            "config [rw]": {
                "description [rw]": "string", 
                "enabled [rw]": "boolean", 
                "mtu [rw]": "uint16", 
                "name [rw]": "string", 
                "type [rw]": "identityref"
            }, 
            "hold_time [rw]": {
                "config [rw]": {
                    "down [rw]": "uint32", 
                    "up [rw]": "uint32"
                }, 
                "state [rw]": {
                    "down [ro]": "uint32", 
                    "up [ro]": "uint32"
                }
            }, 
            "name [rw]": "leafref", 
            "openconfig-if-aggregate:aggregation [rw]": {
                "config [rw]": {
                    "lag_type [rw]": "aggregation-type", 
                    "min_links [rw]": "uint16"
                }, 
                "openconfig-vlan:switched_vlan [rw]": {
                    "config [rw]": {


# Populating models

Now that you have the models loaded, there are different ways to populate them.

## Populating the model programatically

TBD

In [4]:
# We create an interface and set the description and the mtu
et1 = config.interfaces.interface.add("et1")
et1.config.description = "My description"
et1.config.mtu = 1500
print(et1.config.description)
print(et1.config.mtu)

My description
1500


In [5]:
# Let's create a second interface, this time accessing it from the root
config.interfaces.interface.add("et2")
config.interfaces.interface["et2"].config.description = "Another description"
config.interfaces.interface["et2"].config.mtu = 9000
print(config.interfaces.interface["et2"].config.description)
print(config.interfaces.interface["et2"].config.mtu)

Another description
9000


In [6]:
# You can also get the contents as a dict with the ``get`` method.
# ``filter`` let's you decide whether you want to show empty fields or not.
pretty_print(config.get(filter=True))

{
    "interfaces": {
        "interface": {
            "et1": {
                "config": {
                    "description": "My description", 
                    "mtu": 1500
                }, 
                "name": "et1"
            }, 
            "et2": {
                "config": {
                    "description": "Another description", 
                    "mtu": 9000
                }, 
                "name": "et2"
            }
        }
    }
}


In [7]:
# If the value is not valid things will break
try:
    et1.config.mtu = -1
except ValueError as e:
    print(e)

{'error-string': 'mtu must be of a type compatible with uint16', 'generated-type': 'YANGDynClass(base=RestrictedClassType(base_type=int, restriction_dict={\'range\': [\'0..65535\']},int_size=16), is_leaf=True, yang_name="mtu", parent=self, path_helper=self._path_helper, extmethods=self._extmethods, register_paths=True, namespace=\'http://openconfig.net/yang/interfaces\', defining_module=\'openconfig-interfaces\', yang_type=\'uint16\', is_config=True)', 'defined-type': 'uint16'}


Let's work through the interface list:

In [8]:
# Iterating
for iface, data in config.interfaces.interface.items():
    print(iface, data.config.description)

('et1', u'My description')
('et2', u'Another description')


In [9]:
# We can also pop interfaces
print(config.interfaces.interface.keys())
config.interfaces.interface.delete("et1")
print(config.interfaces.interface.keys())

['et1', 'et2']
['et2']


## Populating the model from a dict

You can load a dictionary into the object.

In [10]:
vlans_dict = {
    "vlans": { "vlan": { 100: {
                            "config": {
                                "vlan_id": 100, "name": "production"}},
                         200: {
                            "config": {
                                "vlan_id": 200, "name": "dev"}}}}}
config.load_dict(vlans_dict)
print(config.vlans.vlan.keys())
print(100, config.vlans.vlan[100].config.name)
print(200, config.vlans.vlan[200].config.name)

[200, 100]
(100, u'production')
(200, u'dev')


## Populating the model from a device

You can also load the native configuration of a device into a model.

In [11]:
with eos_device as d:
    running_config = napalm_yang.base.Root()
    running_config.add_model(napalm_yang.models.openconfig_interfaces)
    running_config.parse_config(device=d)

pretty_print(running_config.get(filter=True))

No handlers could be found for logger "napalm-yang"


{
    "interfaces": {
        "interface": {
            "Ethernet1": {
                "config": {
                    "description": "This is a description", 
                    "enabled": True, 
                    "type": "ethernetCsmacd"
                }, 
                "name": "Ethernet1", 
                "routed-vlan": {
                    "ipv4": {
                        "config": {
                            "enabled": False
                        }
                    }
                }
            }, 
            "Ethernet2": {
                "config": {
                    "description": "so much oc", 
                    "enabled": False, 
                    "mtu": 1500, 
                    "type": "ethernetCsmacd"
                }, 
                "name": "Ethernet2", 
                "routed-vlan": {
                    "ipv4": {
                        "addresses": {
                            "address": {
                                "192.168.0.1": {

## Populating from a file

Or from a configuration file stored on disk. The only catch is that you will have to tell `get_config` which profile to use to parse it.

In [12]:
with open("junos.config", "r") as f:
    config = [f.read()]

running_config = napalm_yang.base.Root()
running_config.add_model(napalm_yang.models.openconfig_interfaces)
running_config.parse_config(config=config, profile="junos")

pretty_print(running_config.get(filter=True))

{
    "interfaces": {
        "interface": {
            "ae0": {
                "config": {
                    "description": "ae0", 
                    "enabled": True, 
                    "mtu": 1514, 
                    "name": "ae0", 
                    "type": "ieee8023adLag"
                }, 
                "name": "ae0", 
                "subinterfaces": {
                    "subinterface": {
                        "0": {
                            "config": {
                                "description": "ASDASDASD", 
                                "enabled": True, 
                                "name": "0"
                            }, 
                            "index": "0", 
                            "ipv4": {
                                "addresses": {
                                    "address": {
                                        "172.20.100.1/24": {
                                            "config": {
                                  

# Translating models

Now we know how to populate models, let's translate them into native configuration.

In [13]:
# Let's create a candidate configuration

candidate = napalm_yang.base.Root()
candidate.add_model(napalm_yang.models.openconfig_interfaces())

def create_iface(candidate, name, description, mtu, prefix, prefix_length):
    interface = candidate.interfaces.interface.add(name)
    interface.config.description = description
    interface.config.mtu = mtu
    ip = interface.routed_vlan.ipv4.addresses.address.add(prefix)
    ip.config.ip = prefix
    ip.config.prefix_length = prefix_length

create_iface(candidate, "et1", "Uplink1", 9000, "192.168.1.1", 24)
create_iface(candidate, "et2", "Uplink2", 9000, "192.168.2.1", 24)

pretty_print(candidate.get(filter=True))

{
    "interfaces": {
        "interface": {
            "et1": {
                "config": {
                    "description": "Uplink1", 
                    "mtu": 9000
                }, 
                "name": "et1", 
                "routed-vlan": {
                    "ipv4": {
                        "addresses": {
                            "address": {
                                "192.168.1.1": {
                                    "config": {
                                        "ip": "192.168.1.1", 
                                        "prefix-length": 24
                                    }, 
                                    "ip": "192.168.1.1"
                                }
                            }
                        }
                    }
                }
            }, 
            "et2": {
                "config": {
                    "description": "Uplink2", 
                    "mtu": 9000
                }, 
                "name":

In [14]:
# Now let's translate the object to JunOS

print(candidate.translate_config(profile=junos_device.profile))

<configuration>
  <interfaces>
    <interface>
      <name>et1</name>
      <family>
        <inet>
          <address>
            <name>192.168.1.1/24</name>
          </address>
        </inet>
      </family>
      <description>Uplink1</description>
      <mtu>9000</mtu>
    </interface>
    <interface>
      <name>et2</name>
      <family>
        <inet>
          <address>
            <name>192.168.2.1/24</name>
          </address>
        </inet>
      </family>
      <description>Uplink2</description>
      <mtu>9000</mtu>
    </interface>
  </interfaces>
</configuration>



In [15]:
# And now to EOS

print(candidate.translate_config(eos_device.profile))

interface et1
    ip address 192.168.1.1/24 
    description Uplink1
    mtu 9000
interface et2
    ip address 192.168.2.1/24 
    description Uplink2
    mtu 9000



But this is just the begining, the fun part is yet to come : )

# Advanced manipulation of the configuration

Generating configuration is cool but sometimes is not enough. Let's now see how we can use OpenConfig to make some changes to an existing configuration and generate a "replacement" of the configuration or a "merge".

* A configuration replacement will replace the entire section. For example, if you are manipulating interfaces, all interfaces' configuration will be what's in the model you have. If some configuration parameter is not there, it will be wiped out.
* A configuration merge is a bit different. Lists of elements are synchronized, which means that if you have one or more interfaces in the running configuration that don't exist in the candidate, they will be removed. This will happen to any list of elements. However, elements that exist in both the candidate and the running configuration will be merged, which means that if some attribute is not set in the candidate but is set in the running, it will not be modified.

In [16]:
with junos_device as device:
    # first let's create a candidate config by retrieving the current state of the device
    candidate = napalm_yang.base.Root()
    candidate.add_model(napalm_yang.models.openconfig_interfaces)
    candidate.parse_config(device=junos_device)

    # now let's do a few changes, let's remove lo0.0 and create lo0.1
    candidate.interfaces.interface["lo0"].subinterfaces.subinterface.delete("0")
    lo1 = candidate.interfaces.interface["lo0"].subinterfaces.subinterface.add("1")
    lo1.config.description = "new loopback"

    # Let's also default the mtu of ge-0/0/0 which is set to 1400
    candidate.interfaces.interface["ge-0/0/0"].config._unset_mtu()

    # We will also need a running configuration to compare against
    running = napalm_yang.base.Root()
    running.add_model(napalm_yang.models.openconfig_interfaces)
    running.parse_config(device=junos_device)

In [17]:
# Now let's see how the merge configuration would be
config = candidate.translate_config(profile=junos_device.profile, merge=running)
print(config)

<configuration>
  <interfaces>
    <interface>
      <name>ge-0/0/0</name>
      <unit>
        <name>0</name>
        <family>
          <inet/>
        </family>
        <description>ge-0/0/0.0</description>
      </unit>
      <description>management interface</description>
      <mtu delete="delete"/>
    </interface>
    <interface>
      <name>ge-0/0/1</name>
      <disable/>
      <description>ge-0/0/1</description>
      <mtu>1514</mtu>
    </interface>
    <interface>
      <name>ae0</name>
      <unit>
        <name>0</name>
        <vlan-id>100</vlan-id>
        <family>
          <inet>
            <address>
              <name>192.168.100.1/24</name>
            </address>
            <address>
              <name>172.20.100.1/24</name>
            </address>
          </inet>
        </family>
        <description>ASDASDASD</description>
      </unit>
      <vlan-tagging/>
      <unit>
        <name>1</name>
        <vlan-id>1</vlan-id>
        <family>
          <inet>
 

Note the "delete" tags. Let's actually load the configuration in the device and see which changes are reported.

In [18]:
with junos_device as d:
    d.load_merge_candidate(config=config)
    print(d.compare_config())
    d.discard_config()

[edit interfaces ge-0/0/0]
-   mtu 1400;
[edit interfaces lo0]
-    unit 0 {
-        description lo0.0;
-    }
+    unit 1 {
+        description "new loopback";
+    }


You can see that the device is reporting the changes we expected. Let's try now a replace instead.

In [19]:
config = candidate.translate_config(profile=junos_device.profile, replace=running)
print(config)

<configuration>
  <interfaces replace="replace">
    <interface>
      <name>ge-0/0/0</name>
      <unit>
        <name>0</name>
        <family>
          <inet/>
        </family>
        <description>ge-0/0/0.0</description>
      </unit>
      <description>management interface</description>
    </interface>
    <interface>
      <name>ge-0/0/1</name>
      <disable/>
      <description>ge-0/0/1</description>
      <mtu>1514</mtu>
    </interface>
    <interface>
      <name>ae0</name>
      <unit>
        <name>0</name>
        <vlan-id>100</vlan-id>
        <family>
          <inet>
            <address>
              <name>192.168.100.1/24</name>
            </address>
            <address>
              <name>172.20.100.1/24</name>
            </address>
          </inet>
        </family>
        <description>ASDASDASD</description>
      </unit>
      <vlan-tagging/>
      <unit>
        <name>1</name>
        <vlan-id>1</vlan-id>
        <family>
          <inet>
            

Note that instead of "delete", now we have a replace in one of the top containers, indicating to the device we want to replace everything underneath. Let's merge and see what happens:

In [20]:
with junos_device as d:
    d.load_merge_candidate(config=config)
    print(d.compare_config())
    d.discard_config()

[edit interfaces ge-0/0/0]
-   mtu 1400;
[edit interfaces ge-0/0/0 unit 0 family inet]
-       dhcp;
[edit interfaces lo0]
-    unit 0 {
-        description lo0.0;
-    }
+    unit 1 {
+        description "new loopback";
+    }


Interestingly, there is an extra change. That is due to the fact that the `dhcp` parameter is outside our model's control.

### Not so friendly platforms

This also works with not so friendly platforms. Let's do the same we did in the previous section with an EOS device.

In [ ]:
with eos_device as device:
    # first let's create a candidate config by retrieving the current state of the device
    candidate = napalm_yang.base.Root()
    candidate.add_model(napalm_yang.models.openconfig_interfaces)
    candidate.parse_config(device=device)

    # now let's do a few changes, let's remove lo1 and create lo0
    candidate.interfaces.interface.delete("Loopback1")
    lo0 = candidate.interfaces.interface.add("Loopback0")
    lo0.config.description = "new loopback"

    # Let's also default the mtu of ge-0/0/0 which is set to 1400
    candidate.interfaces.interface["Port-Channel1"].config._unset_mtu()

    # We will also need a running configuration to compare against
    running = napalm_yang.base.Root()
    running.add_model(napalm_yang.models.openconfig_interfaces)
    running.parse_config(device=device)

In [ ]:
# Now let's see how the merge configuration would be
config = candidate.translate_config(profile=eos_device.profile, merge=running)
print(config)

In [23]:
with eos_device as d:
    d.load_merge_candidate(config=config)
    print(d.compare_config())
    d.discard_config()

@@ -19,7 +19,6 @@
 !
 interface Port-Channel1
    description blah
-   mtu 9000
    no switchport
 !
 interface Port-Channel1.1
@@ -47,8 +46,10 @@
    encapsulation dot1q vlan 2
    ip address 192.168.2.1/24
 !
+interface Loopback0
+   description new loopback
+!
 interface Loopback1
-   description a loopback
 !
 interface Management1
    description "oob blah"


As in the previous example, we got exactly the same changes we were expecting.

In [25]:
config = candidate.translate_config(profile=eos_device.profile, replace=running)
print(config)

default interface Port-Channel1
interface Port-Channel1
    description blah
no interface Port-Channel1.1
interface Port-Channel1.1
    description "prod uplink"
default interface Ethernet1
interface Ethernet1
    description This is a description
default interface Ethernet2
interface Ethernet2
    ip address 192.168.0.1/24 
    shutdown
    description so much oc
    mtu 1500
no interface Ethernet2.1
interface Ethernet2.1
    encapsulation dot1q vlan 1
    ip address 192.168.1.1/24 
    ip address 172.20.0.1/24 secondary
    description another subiface
no interface Ethernet2.2
interface Ethernet2.2
    encapsulation dot1q vlan 2
    ip address 192.168.2.1/24 
    description asdasdasd
default interface Management1
interface Management1
    ip address 10.0.2.15/24 
    description "oob blah"
    mtu 1500
default interface Loopback0
interface Loopback0
    description new loopback
default interface Loopback1



In [26]:
with eos_device as d:
    d.load_merge_candidate(config=config)
    print(d.compare_config())
    d.discard_config()

@@ -19,21 +19,16 @@
 !
 interface Port-Channel1
    description blah
-   mtu 9000
-   no switchport
 !
 interface Port-Channel1.1
    description "prod uplink"
 !
 interface Ethernet1
    description This is a description
-   dcbx mode ieee
-   channel-group 1 mode active
 !
 interface Ethernet2
    description so much oc
    shutdown
-   no switchport
    ip address 192.168.0.1/24
 !
 interface Ethernet2.1
@@ -47,8 +42,10 @@
    encapsulation dot1q vlan 2
    ip address 192.168.2.1/24
 !
+interface Loopback0
+   description new loopback
+!
 interface Loopback1
-   description a loopback
 !
 interface Management1
    description "oob blah"


With the replace instead, we got some extra changes as some things are outside our model's control.

### Generate config, merge or replace

Which of the three methods to choose is very subjective and it will depend on your operations:

* Generating configuration. The drawback of this one is that configuration is only applied, never removed so it's good for places where you don't know or control most of your configuration.
* "Merge" configuration. This one is good to keep some configuration in a known state while leaving other outside it.
* "Replace" configuration. This one allows you to fully control the configuration. If you can use this, it means you dictate the fate of your network and not the other way around.

# Diffing objects

Right now we have seen we can rely on the on-box diff to see the changes to the device. However, you might want to diff the objects directly in certain cases. You can do that with the ``diff`` method. Note that the method will tell you only which changes are to be performed for the models that are known to your binding.

In [26]:
diff = candidate.diff(running)
pretty_print(diff)

{
    "interfaces": {
        "interface": {
            "both": {
                "Port-Channel1": {
                    "config": {
                        "mtu": {
                            "mine": null, 
                            "other": 9000
                        }
                    }
                }
            }, 
            "mine": {
                "Loopback0": {
                    "config": {
                        "description": "new loopback"
                    }
                }
            }, 
            "other": {
                "Loopback1": {
                    "config": {
                        "description": "a loopback", 
                        "enabled": true, 
                        "mtu": 1500, 
                        "name": "Loopback1", 
                        "type_": "softwareLoopback"
                    }, 
                    "name": "Loopback1", 
                    "routed_vlan": {
                        "ipv4": {
                